In [2]:
import os
import re
import random
import gensim
import numpy as np
import pandas as pd
import nltk
from operator import setitem
from six import iteritems
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics
from sklearn.cluster import SpectralClustering
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import cluster
import h5py
from sklearn.manifold import MDS
MDS()

C:\Users\ifue3702\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


MDS(dissimilarity='euclidean', eps=0.001, max_iter=300, metric=True,
  n_components=2, n_init=4, n_jobs=1, random_state=None, verbose=0)

In [1]:
from Litho.nlp_funcs import *
from Litho.similarity import (check_similarity, match_lithcode, jaccard_similarity, 
                              calc_similarity_score, print_sim_compare, merge_similar_words)


C:\Users\ifue3702\AppData\Local\Continuum\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


ImportError: No module named 'ParsingAndCleaningPapers2BuildDoc2VecModel'

In [4]:
from sklearn.manifold import TSNE
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, HoverTool


def plot_embedding(arr, labels=None, filename='plot.html'):
    """Short summary.

    Parameters
    ----------
    arr : np.array
       2D array with embeddings.
    labels : list(str)
       List of labels (the default is None).
    filename : str
       Path to the output html file containing the plot (the default is 'plot.html').

    """
    tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=300)
    tsne_results = tsne.fit_transform(arr)

    plot_data = {
       'x': tsne_results[:, 0],
       'y': tsne_results[:, 1]
    }

    plot_tools = ['pan', 'zoom_in', 'zoom_out']

    if labels is not None:
        plot_data['label'] = labels

        hover = HoverTool(tooltips=[
           ("Description", "@label")
        ])

        plot_tools.append(hover)

    to_plot = ColumnDataSource(plot_data)
    p = figure(title="Descriptions t-SNE", tools=plot_tools)
    p.xaxis.axis_label = ''
    p.yaxis.axis_label = ''

    p.circle('x', 'y', source=to_plot,
            fill_alpha=0.8, size=10)

    # p.add_layout(labels)
    output_file(filename)
    show(p)
    
# plot_embedding([doc2vec.infer_vector(x) for x in lemmatized[:1000]],
#                [';'.join(x) for x in lemmatized[:1000]])

In [5]:
def labeledSentences(tokenizedDescriptions, labels = None):
    sentences = [gensim.models.doc2vec.TaggedDocument(words = i, tags = [labels[n]]) for n, i in enumerate(tokenizedDescriptions)]
    return sentences

In [6]:
root = os.getcwd()
file = 'reclasified.csv'

DF = pd.read_csv(os.path.join(root,file))

In [7]:
stopa = nltk.corpus.stopwords.words('english')

In [8]:
stopw2 = ['redish', 'reddish', 'red', 'black', 'blackish', 'brown', 'brownish',
          'blue', 'blueish', 'orange', 'orangeish', 'gray', 'grey', 'grayish',
          'greyish', 'white', 'whiteish', 'purple', 'purpleish', 'yellow',
          'yellowish', 'green', 'greenish', 'light', 'very', 'pink', 'coloured', 'multicoloured',
          'dark', 'color', 'colour', 'hard', 'soft', 'water', 'supply', 'fine', 'coarse',
          'medium', 'bearing', 'pipe', 'sticky', 'tough', 'small', 'stiff', 
          'running', 'streaks', 'nominal', 'bands', 'back', 'slippery', 'loose', 
          'broken', 'fractured', 'surface', 'surface', 'rotten', 'compacted', 'seams',
          'dry', 'wet', 'cemented', 'rock', 'trap', 'ridge', 'large', 'small', 'thin', 'soak',
          'creek', 'ridge', 'clean', 'decomposed', 'band']


stopa= stopa+stopw2  # add the additional stopwords above

In [9]:
# stopw2 = ['redish', 'reddish', 'red', 'black', 'blackish', 'brown', 'brownish',
#           'blue', 'blueish', 'orange', 'orangeish', 'gray', 'grey', 'grayish',
#           'greyish', 'white', 'whiteish', 'purple', 'purpleish', 'yellow',
#           'yellowish', 'green', 'greenish', 'light', 'very', 'pink', 'coloured', 'multicoloured',
#           'dark', 'color', 'colour']
# stopa= stopa+stopw2  # add the additional stopwords above

## doc2vec model from descriptions (train-validation subsets)

In [10]:
msk = np.random.rand(len(DF)) < 0.8
train = DF[msk]
validation = DF[~msk]


In [11]:
train = train.dropna(how = 'any')
Tags = train['OWN'].tolist()
print(len(Tags))
Descriptions=train.Description.tolist()
Descriptions = [str(n) for n in Descriptions]

#use extend so it's a big flat list of vocab
totalvocab_lemmatized = []
totalvocab_tokenized = []
for i in Descriptions:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    totalvocab_lemmatized.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    totalvocab_tokenized.extend([allwords_tokenized])

print(np.shape(totalvocab_tokenized),  np.shape(totalvocab_lemmatized))
Sentences = totalvocab_tokenized
Sentences

102167
(102167,) (102167,)


[['sand'],
 ['clay'],
 ['clay'],
 ['clay', 'sandy'],
 ['drift'],
 ['sandstone'],
 ['sandstone'],
 ['clay', 'sandy'],
 ['clay'],
 ['clay', 'sandy'],
 ['sand', 'drift'],
 ['sand'],
 ['clay', 'sand'],
 ['clay', 'boulder'],
 ['sandstone'],
 ['clay'],
 ['clay', 'drift'],
 ['clay'],
 ['clay'],
 ['clay'],
 ['clay'],
 ['sand', 'drift'],
 ['clay'],
 ['clay'],
 ['drift'],
 ['clay'],
 ['drift', 'stoney'],
 ['drift'],
 ['cement'],
 ['clay', 'sand'],
 ['sand', 'drift'],
 ['clay'],
 ['limestone'],
 ['pebbles'],
 ['clay'],
 ['clay'],
 ['loam'],
 ['clay'],
 ['clay'],
 ['clay'],
 ['sandstone'],
 ['sand'],
 ['sandstone'],
 ['sandstone'],
 ['sand', 'silt'],
 ['silt'],
 ['shale'],
 ['sandstone'],
 ['sand'],
 ['sandstone'],
 ['clay'],
 ['sandstone', 'drift'],
 ['shale'],
 ['stones'],
 ['clay'],
 ['sandstone'],
 ['sand', 'drift'],
 ['sandstone'],
 ['loam', 'sandy'],
 ['clay'],
 ['sand', 'drift'],
 ['quartz'],
 ['clay'],
 ['clay'],
 ['clay'],
 ['clay'],
 ['sandstone'],
 ['clay'],
 ['sandstone'],
 ['sandstone

In [ ]:
labSen =  labeledSentences(Sentences, labels = Tags)
model = gensim.models.Doc2Vec(size = 200,
                             min_count = 1,
                             workers = 8,
                             iter = 100)
model.build_vocab(labSen)
#training of model
for epoch in range(20):
    print('iteration '+str(epoch+1))
    model.train(labSen, total_examples=len(labSen), epochs = 20)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

In [ ]:
validation = validation.dropna(how = 'any')
# classes = validation['OWN'].tolist()

text=validation.Description.tolist()
text = [str(n) for n in text]

#use extend so it's a big flat list of vocab
lemmatized = []
tokenized = []
for i in text:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    lemmatized.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    tokenized.extend([allwords_tokenized])

print(np.shape(tokenized),  np.shape(lemmatized))
texts = tokenized

In [ ]:
vectors = []
for n in tokenized:
    vectors.append(np.reshape(model.infer_vector(n), (1,200)))
validation['vectors'] = vectors

X = np.vstack(validation.vectors)
kmeans = cluster.MiniBatchKMeans(n_clusters = len(validation.OWN.unique().tolist())).fit(X)
pred_classes = kmeans.predict(X)
print(metrics.adjusted_rand_score( validation['OWN'].tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score( validation['OWN'].tolist(), pred_classes.tolist()), 
      metrics.completeness_score( validation['OWN'].tolist(), pred_classes.tolist()))

## Dictionary building from descriptions

In [8]:
DF = DF[['Description']]
DF = DF.dropna(how = 'any')
Descriptions=DF.Description.tolist()
Descriptions = [str(n) for n in Descriptions]

#use extend so it's a big flat list of vocab
totalvocab_lemmatized = []
totalvocab_tokenized = []
for i in Descriptions:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    totalvocab_lemmatized.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    totalvocab_tokenized.extend([allwords_tokenized])

print(np.shape(totalvocab_tokenized),  np.shape(totalvocab_lemmatized))

(127930,) (127930,)


In [13]:
###here just tokens or stemmed tokens can be used alternatively
dictionary = gensim.corpora.Dictionary(totalvocab_lemmatized)
## newt two lines remove tokens contained just once in the dictionary
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(once_ids)

dictionary.compactify()
print(dictionary)


Dictionary(1205 unique tokens: ['gyre', 'cobble', 'lava', 'drilling', 'limit']...)


In [14]:
#use extend so it's a big flat list of vocab
Namoi = 'reclasified.csv'
Data = pd.read_csv(os.path.join(root,Namoi))
Data = Data.dropna(subset=['Description'])
Descriptions = Data[['Description']]
Descriptions=Descriptions.Description.tolist()
Descriptions = [str(n) for n in Descriptions]
lemm_total = []
token_total = []
for i in Descriptions:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    lemm_total.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    token_total.extend([allwords_tokenized])

print(np.shape(token_total),  np.shape(lemm_total))
Data['words'] = [len(n) for n in token_total]
Data['Tokens'] = token_total
Data['Lemmas'] = lemm_total
Data = Data[Data.words > 0]
corpus = [dictionary.doc2bow(text) for text in Data.Tokens.tolist()]
print('length corpus', len(corpus))

(127930,) (127930,)
length corpus 127626


In [96]:
len(Data.second.unique())

26

In [15]:
token_total = [n for n in token_total if len(n)>0]
lemm_total = [n for n in lemm_total if len(n)>0]
print(len(lemm_total), len(token_total))

127627 127626


In [23]:
DataSampled = Data.sample(1000)
DataDes=DataSampled.Description.tolist()
DataDes = [str(n) for n in DataDes]


# #use extend so it's a big flat list of vocab
lemmatized = []
tokenized = []
for i in DataDes:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    lemmatized.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    tokenized.extend([allwords_tokenized])

print(np.shape(tokenized),  np.shape(lemmatized))


(1000,) (1000,)


## TF-IDF LSI and LDA models

In [16]:
%time tfidf = gensim.models.TfidfModel(corpus, normalize = True)
corpus_tfidf = tfidf[corpus]

Wall time: 372 ms


In [18]:
def vectorSimilarity(Model, dictio, MatrixSim, text):
    vec_bow = dictio.doc2bow(text)
    vec_tfidf = Model[vec_bow]
    vectSim = MatrixSim[vec_tfidf]
    if str(type(vectSim)) == "<class 'numpy.ndarray'>":
        return vectSim
    else:
        return np.asarray(vectSim)
    

In [24]:
vectors = []
for n, i in enumerate(tokenized):
    vec_bow = dictionary.doc2bow(i)
    vec_tfidf = tfidf[vec_bow]
    vector = gensim.matutils.sparse2full(vec_tfidf, length = len(corpus))
    vectors.append(vector)

In [103]:
DataSampled['vecs'] = vectors
X = np.vstack(DataSampled.vecs)
kmeans = MiniBatchKMeans(n_clusters = len(Data.second.unique()), batch_size = 100).fit(X)
pred_classes = kmeans.predict(X)

print(metrics.adjusted_rand_score(DataSampled.second.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(DataSampled.second.tolist(), pred_classes.tolist()), 
      metrics.completeness_score(DataSampled.second.tolist(), pred_classes.tolist()))

0.695522383088 0.711751123329 0.73052580772


In [101]:
lsi = gensim.models.LsiModel(corpus, id2word=dictionary, num_topics=300)

INFO:gensim.models.lsimodel:using serial LSI version on this node
INFO:gensim.models.lsimodel:updating model with new documents
INFO:gensim.models.lsimodel:preparing a new chunk of documents
INFO:gensim.models.lsimodel:using 100 extra samples and 2 power iterations
INFO:gensim.models.lsimodel:1st phase: constructing (1148, 400) action matrix
INFO:gensim.models.lsimodel:orthonormalizing (1148, 400) action matrix
INFO:gensim.models.lsimodel:2nd phase: running dense svd on (400, 20000) matrix
INFO:gensim.models.lsimodel:computing the final decomposition
INFO:gensim.models.lsimodel:keeping 176 factors (discarding 0.000% of energy spectrum)
INFO:gensim.models.lsimodel:processed documents up to #20000
INFO:gensim.models.lsimodel:topic #0(87.702): 0.942*"clay" + 0.230*"gravel" + 0.227*"sandy" + 0.084*"sand" + 0.031*"shale" + 0.015*"soil" + 0.015*"sandstone" + 0.011*"puggy" + 0.007*"stoney" + 0.007*"solid"
INFO:gensim.models.lsimodel:topic #1(57.330): 0.916*"shale" + 0.245*"sandy" + -0.245*"gr

In [102]:
vectors = []
for n, i in enumerate(tokenized):
    vec_bow = dictionary.doc2bow(i)
    vec_lsi = lsi[vec_bow]
    vector = gensim.matutils.sparse2full(vec_lsi, length = 300)
    vectors.append(vector)

In [93]:
plot_embedding([x for x in vectors[:1000]],
               [';'.join(x) for x in tokenized[:1000]])

INFO:bokeh.core.state:Session output file 'plot.html' already exists, will be overwritten.


In [65]:
lda = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, num_topics=50, chunksize=10000, passes=2, workers = 8)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.02
INFO:gensim.models.ldamodel:using symmetric eta at 0.0008710801393728223
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running online LDA training, 50 topics, 2 passes over the supplied corpus of 127626 documents, updating every 80000 documents, evaluating every ~127626 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamulticore:training LDA model using 8 processes
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/127626, outstanding queue size 1
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/127626, outstanding queue size 2
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #2 = documents up to #30000/127626, outstanding queue size 3
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #3 = documents up to #40000/12762

In [66]:
vectors = []
for n, i in enumerate(tokenized):
    vec_bow = dictionary.doc2bow(i)
    vec_lda = lda[vec_bow]
    vector = gensim.matutils.sparse2full(vec_lda, length = 50)
    vectors.append(vector)

### Dictionary from papers

In [17]:
tfidfSim = gensim.similarities.Similarity(root+'/tmp/',corpus_tfidf, len(dictionary))

In [21]:
vector =vectorSimilarity(tfidf, dictionary, tfidfSim, lemm_total[0])
n_rows = vector.shape[0]
chunck_size = (1000, 1000)


with h5py.File('tmp/tfidf.h5', 'w') as f:
    dset= f.create_dataset('chunked', shape = (n_rows, n_rows), chunks = chunck_size)
    for n, i in enumerate(lemm_total):
        dset[n, :] = vectorSimilarity(tfidf, dictionary, tfidfSim, i)

#     last_row = dset[-1, :].copy()
    
# # Read file
# test = h5py.File('tmp/test.h5', 'r')
# data = test['chunked']
# np.all(data[-1, :] == last_row)
    

In [22]:
# Read file
test = h5py.File('tmp/tfidf.h5', 'r')
data = test['chunked']


In [25]:
import dask.array as da
darray = da.from_array(data, chunks=(1000, 1000))


In [28]:
mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)


In [ ]:
pos = mds.fit_transform(darray)  # shape (n_components, n_samples)


In [ ]:
n_rows = 100
chuck_size = (10, 10)  # Cambia esto a (100, 100) o algo así
with h5py.File('/tmp/test.h5', 'w') as f:
    dset = f.create_dataset('data', (n_rows, n_rows), chunks=chuck_size)
    for i in range(n_rows):
        dset[i, :] = similarity()  # Acá es donde tienes que calcular la similarity
    last_row = dset[-1, :].copy()
 
# Read file
test = h5py.File('/tmp/test.h5', 'r')
data = test['data']
np.all(data[-1, :] == last_row)

In [84]:
vec_bow = dictionary.doc2bow(lemm_total[0])
vec_tfidf = tfidf[vec_bow]
sims1 = tfidfSim[vec_tfidf]

dtype = sims1.dtype
print(sims1.shape, dtype)

(127930,) float32


In [69]:
# tfidfSimMatrix =[]

simsss = []
# vectors = []
#with h5py.File('tmp/tfidf.hdf5'.format(n), 'w') as f:
for n, i in enumerate(lemm_total[:30]):
    print(i)
#         print(i, n)
    vec_bow = dictionary.doc2bow(i)
#         print(vec_bow)
    vec_tfidf = tfidf[vec_bow]
    print(vec_tfidf)
#    simsss.append(gensim.matutils.corpus2dense(vec_tfidf, num_terms=300))
#         print(vec_tfidf)
        
#     original = [0]*len(dictionary)
#     replaceIndex = [n[0] for n in vec_tfidf]
#     replaceValue = [n[1] for n in vec_tfidf]
#     for b, c in zip(replaceIndex, replaceValue):
#         setitem(original, b, c)
#     vectors.append(original)


#        sims1 = tfidfSim[vec_tfidf]
    
    
#         print(len(sims1))
#         simsss.append(sims1)


#        f.create_dataset('tfidf_{}'.format(n), data = sims1)
    
    
#     if (n>0) & (n%10 == 0):
#         simsss = np.asarray(simsss)
#        
#             f.create_dataset('tfidf_{}'.format(n), data = simsss)
#             print(f.shape)
#             f.close()
            
#         simsss = []
#     elif n == len(lemm_total):
#         simsss = np.asarray(simsss)
#         with h5py.File('tmp/tfidf_{}.hdf5'.format(n), 'w') as f:
#             f.create_dataset('tfidf_{}'.format(n), data = simsss)
#             print(f.shape)
#             f.close()
            
#         simsss = []
    
# simsss.shape
#     print(sims1)
#     tfidfSimMatrix.append(sims1)
# tfidfDisMatrix = 1- np.array(tfidfSimMatrix) 


#f.close()
#print(len(simsss), simsss[0])                

['sand']
[(1047, 1.0)]
['clay']
[(217, 1.0)]
['drift']
[(290, 1.0)]
['clay', 'sand']
[(217, 0.44255164661160346), (1047, 0.8967430178603893)]
['clay']
[(217, 1.0)]
['clay', 'sandy']
[(217, 0.35543402011792474), (1013, 0.9347013733502325)]
['drift']
[(290, 1.0)]
['sand', 'drift']
[(290, 0.9636659480506097), (1047, 0.26711035279022716)]
['sandstone']
[(834, 1.0)]
['sandstone']
[(834, 1.0)]
['sandstone']
[(834, 1.0)]
['clay', 'sandy']
[(217, 0.35543402011792474), (1013, 0.9347013733502325)]
['clay']
[(217, 1.0)]
['clay', 'sandy']
[(217, 0.35543402011792474), (1013, 0.9347013733502325)]
['sand', 'drift']
[(290, 0.9636659480506097), (1047, 0.26711035279022716)]
['sand']
[(1047, 1.0)]
['clay', 'sand']
[(217, 0.44255164661160346), (1047, 0.8967430178603893)]
['clay', 'boulder']
[(217, 0.1698892421345605), (799, 0.9854631628868451)]
['sandstone']
[(834, 1.0)]
['clay']
[(217, 1.0)]
['clay', 'drift']
[(217, 0.13552969245277532), (290, 0.9907732850978855)]
['clay', 'pebble', 'pebbly']
[(217, 0.10

In [72]:
numpy_matrix = np.random.randint(10, size=[5,2])  # random matrix as an example
print(numpy_matrix)
corpus = gensim.matutils.Dense2Corpus(numpy_matrix)
print(corpus)
numpy_matrix = gensim.matutils.corpus2dense(corpus, num_terms=5)
numpy_matrix

[[0 0]
 [1 4]
 [6 8]
 [6 9]
 [5 8]]


array([[ 0.,  0.],
       [ 1.,  4.],
       [ 6.,  8.],
       [ 6.,  9.],
       [ 5.,  8.]], dtype=float32)

In [60]:
file = 'tfidf.hdf5'
check = h5py.File(os.path.join(root,os.path.join('tmp',file)), 'r')
for n in check:
    print(check[str(n)])
check['tfidf_2']

<HDF5 dataset "tfidf_0": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_1": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_10": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_11": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_12": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_13": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_14": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_15": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_16": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_17": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_18": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_19": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_2": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_20": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_21": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_22": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_23": shape (127930,), type "<f4">
<HDF5 dataset "tfidf_24": shape (127930,), type "<f4">
<HDF5 dataset

<HDF5 dataset "tfidf_2": shape (127930,), type "<f4">

In [16]:
test = range(0,100)
for n in test:
    if n%10 == 0:
        print(n)

0
10
20
30
40
50
60
70
80
90


In [106]:
# DataSampled['sim'] = simsss
DataSampled['vecs'] = vectors
X = np.vstack(DataSampled.vecs)
kmeans = MiniBatchKMeans(n_clusters = 90, batch_size = 100).fit(X)
pred_classes = kmeans.predict(X)

print(metrics.adjusted_rand_score(DataSampled.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(DataSampled.OWN.tolist(), pred_classes.tolist()), 
      metrics.completeness_score(DataSampled.OWN.tolist(), pred_classes.tolist()))

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [165]:
lsi = gensim.models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=300)

INFO:gensim.models.lsimodel:using serial LSI version on this node
INFO:gensim.models.lsimodel:updating model with new documents
INFO:gensim.models.lsimodel:preparing a new chunk of documents
INFO:gensim.models.lsimodel:using 100 extra samples and 2 power iterations
INFO:gensim.models.lsimodel:1st phase: constructing (1205, 400) action matrix
INFO:gensim.models.lsimodel:orthonormalizing (1205, 400) action matrix
INFO:gensim.models.lsimodel:2nd phase: running dense svd on (400, 20000) matrix
INFO:gensim.models.lsimodel:computing the final decomposition
INFO:gensim.models.lsimodel:keeping 240 factors (discarding 0.000% of energy spectrum)
INFO:gensim.models.lsimodel:processed documents up to #20000
INFO:gensim.models.lsimodel:topic #0(52.220): 0.936*"clay" + 0.194*"gravel" + 0.191*"sandy" + 0.115*"supply" + 0.109*"water" + 0.091*"yellow" + 0.074*"shale" + 0.072*"sand" + 0.051*"sandstone" + 0.030*"grey"
INFO:gensim.models.lsimodel:topic #1(44.171): -0.675*"shale" + -0.418*"supply" + -0.382

In [166]:
lsiSim = gensim.similarities.Similarity(root+'/tmp/',lsi[corpus_tfidf], len(dictionary))
# vectors =[]
simsss = []
for n in lemmatized[:10]:
    vec_bow = dictionary.doc2bow(n)
    vec_lsi = lsi[vec_bow]
    sims1 = lsiSim[vec_lsi]
    simsss.append(sims1)
    
#     vecs.append(vec_lsi)


INFO:gensim.similarities.docsim:starting similarity index under C:\Users\ifue3702\Documents\GitHub\Litho/tmp/
INFO:gensim.similarities.docsim:PROGRESS: fresh_shard size=10000
INFO:gensim.similarities.docsim:PROGRESS: fresh_shard size=20000
INFO:gensim.similarities.docsim:PROGRESS: fresh_shard size=30000
INFO:gensim.similarities.docsim:creating sparse index
INFO:gensim.matutils:creating sparse matrix from corpus
INFO:gensim.matutils:PROGRESS: at document #0/32768
INFO:gensim.matutils:PROGRESS: at document #10000/32768
INFO:gensim.matutils:PROGRESS: at document #20000/32768
INFO:gensim.matutils:PROGRESS: at document #30000/32768
INFO:gensim.similarities.docsim:created <32768x1205 sparse matrix of type '<class 'numpy.float32'>'
	with 9826486 stored elements in Compressed Sparse Row format>
INFO:gensim.similarities.docsim:creating sparse shard #0
INFO:gensim.similarities.docsim:saving index shard to C:\Users\ifue3702\Documents\GitHub\Litho/tmp\.0
INFO:gensim.utils:saving SparseMatrixSimila

In [87]:
plot_embedding([x for x in simsss[:1000]],
               [';'.join(x) for x in lemmatized[:1000]])

INFO:bokeh.core.state:Session output file 'plot.html' already exists, will be overwritten.


In [82]:
lda = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, num_topics=50, chunksize=10000, passes=10, workers = 8)

In [85]:
ldaSim = gensim.similarities.Similarity(root+'/tmp/', lda[corpus], len(dictionary))
# vectors =[]
simsss = []
for n in lemmatized:
    vec_bow = dictionary.doc2bow(n)
    vec_lda = lda[vec_bow]
    sims1 = ldaSim[vec_lda]
    simsss.append(sims1)

INFO:gensim.similarities.docsim:starting similarity index under C:\Users\ifue3702\Documents\GitHub\Litho/tmp/
INFO:gensim.similarities.docsim:PROGRESS: fresh_shard size=10000
INFO:gensim.similarities.docsim:PROGRESS: fresh_shard size=20000
INFO:gensim.similarities.docsim:PROGRESS: fresh_shard size=30000
INFO:gensim.similarities.docsim:creating sparse index
INFO:gensim.matutils:creating sparse matrix from corpus
INFO:gensim.matutils:PROGRESS: at document #0/32768
INFO:gensim.matutils:PROGRESS: at document #10000/32768
INFO:gensim.matutils:PROGRESS: at document #20000/32768
INFO:gensim.matutils:PROGRESS: at document #30000/32768
INFO:gensim.similarities.docsim:created <32768x1205 sparse matrix of type '<class 'numpy.float32'>'
	with 57966 stored elements in Compressed Sparse Row format>
INFO:gensim.similarities.docsim:creating sparse shard #0
INFO:gensim.similarities.docsim:saving index shard to C:\Users\ifue3702\Documents\GitHub\Litho/tmp\.0
INFO:gensim.utils:saving SparseMatrixSimilari

In [408]:
size = 1000
labs = list(range(len(totalvocab_lemmatized)))
labs = ['lab_{}'.format(n) for n in labs]
taggedDescriptions = labeledSentences(totalvocab_lemmatized, labels = labs)
doc2vec = gensim.models.Doc2Vec(size = size, window = 13, workers = 8, seed = 1, 
                                min_count = 5, sample = 0.1e-5, dm = 1, dm_concat = 1, 
                                dm_mean = 1, alpha = 0.25)
doc2vec.build_vocab(taggedDescriptions)


In [409]:
vectors = []
for n in lemmatized:
    vectors.append(np.reshape(doc2vec.infer_vector(n), (1,size)))
vectors = np.reshape(np.asarray(vectors), (1000, size))
kmeans = MiniBatchKMeans(n_clusters = 90, batch_size = 100).fit(vectors)
pred_classes = kmeans.predict(vectors)

print(metrics.adjusted_rand_score(DataSampled.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(DataSampled.OWN.tolist(), pred_classes.tolist()),
      metrics.completeness_score(DataSampled.OWN.tolist(), pred_classes.tolist()))

0.434457623617 0.608323503602 0.665401278107


In [43]:
from nltk.corpus import wordnet as wn

In [233]:
clay = wn.synset('clay.n.01')
mud = wn.synset('mud.n.01')
# pug = wn.synset('pug.n.02')
sand = wn.synset('sand.n.01')
silt = wn.synset('silt.n.01')
gravel =  wn.synset('gravel.n.01')
stone = wn.synset('stone.n.01')
pebble = wn.synset('pebble.n.01')
boulder = wn.synset('boulder.n.01')
granite = wn.synset('granite.n.01')
basalt = wn.synset('basalt.n.01')
lava = wn.synset('lava.n.01')
tuff = wn.synset('tuff.n.01')
volcanic = wn.synset('volcanic.s.03')
rhyolite = wn.synset('rhyolite.n.01')
breccia = wn.synsets('breccia')[0]
agglomerate = wn.synset('agglomerate.n.01')
zeolite = wn.synset('zeolite.n.01')
andesite = wn.synset('andesite.n.01')
diorite = wn.synset('diorite.n.01')
porphyry = wn.synset('porphyry.n.01')
# dolerite = wn.synset('dolerite.n.01')
feldspar = wn.synset('feldspar.n.01')
igneous = wn.synset('igneous.a.01')

sandstone = wn.synset('sandstone.n.01')
# greywacke = wn.synset('greywacke.n.01')
# arkose = wn.synset('arkose.n.01')

shale = wn.synset('shale.n.01')
claystone = wn.synset('claystone.n.01')
# mudstone = wn.synset('mudstone.n.01')
siltstone = wn.synset('siltstone.n.01')

limestone =  wn.synset('limestone.n.01')
dolomite = wn.synset('dolomite.n.01')
# calcrete = wn.synset('calcrete.n.01')
siderite = wn.synset('siderite.n.01')
chalk = wn.synset('chalk.n.01')
marl = wn.synset('marl.n.01')
calcite = wn.synset('calcite.n.01')

mica = wn.synset('mica.n.01')
schist = wn.synset('schist.n.01')
# serpentine = wn.synset('serpentine.n.01')
soapstone = wn.synset('soapstone.n.01')
slate = wn.synset('slate.n.03')
# phyllite  = wn.synset('phyllite.n.01')

carbonaceous = wn.synset('carbonaceous.a.01')
coal = wn.synset('coal.n.01')
lignite = wn.synset('lignite.n.01')
wood = wn.synset('wood.n.01')

jasper = wn.synset('jasper.n.01')
chert = wn.synset('chert.n.01')
# silcrete = wn.synset('silcrete.n.01')
laterite = wn.synset('laterite.n.01')
# ironstone = wn.synset('ironstone.n.01')
cement = wn.synset('cement.n.03')
# blueMetal = wn.synset('wood.n.01')

soil = wn.synset('soil.n.02')
loam = wn.synset('loam.n.01')
wood = wn.synset('wood.n.01')

bentonite = wn.synset('bentonite.n.01')

conglomerate = wn.synset('conglomerate.n.01')

drift = wn.synset('drift.n.04')

quartz = wn.synset('quartz.n.02')

rock = wn.synset('rock.n.01')
alluvium = wn.synset('alluvium.n.01')
water = wn.synset('water.n.01')
cavity = wn.synset('cavity.n.01')
sedimentary = wn.synset('sedimentary.a.01')
channel = wn.synset('channel.n.02')
ground = wn.synset('ground.n.07')
earth = wn.synset('earth.n.02')

sandy = wn.synsets('sandy', pos = wn.ADJ)[1]
clayey = wn.synsets('clayey', pos = wn.ADJ)[0]
silty = wn.synsets('silty')[0]
gravelly = wn.synset('gravelly.s.01')
stony = wn.synsets('stony')[0]
pebbly = wn.synsets('pebbly')[0]
gritty = wn.synsets('gritty')[0]



# for n in drift:
#     print(n.definition())
print(earth)
print(earth.definition())
# sandy = wngreywacke.synset('sandy.a.01')
# for n in sandy:
#     print(n.definition())
# print(sandy)

Synset('earth.n.02')
the loose soft material that makes up a large part of the land surface


In [176]:

Lemma = nltk.WordNetLemmatizer()
print(Lemma.lemmatize('carbonaceous'))

carbonaceous


In [247]:
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer()
tfidf = vect.fit_transform([n.lower() for n in Descriptions])

In [248]:
kmeans = MiniBatchKMeans(n_clusters = 90, batch_size = 100, ).fit(tfidf)
pred_classes = kmeans.predict(tfidf)

print(metrics.completeness_score(Data.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_rand_score(Data.OWN.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(Data.OWN.tolist(), pred_classes.tolist()))

0.404928867022 0.232049174226 0.401791987721


In [412]:
from __future__ import division
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import brown
import math
import numpy as np
import sys


In [413]:
# Parameters to the algorithm. Currently set to values that was reported
# in the paper to produce "best" results.
ALPHA = 0.2
BETA = 0.45
ETA = 0.4
PHI = 0.2
DELTA = 0.85

brown_freqs = dict()
N = 0

In [450]:
######################### word similarity ##########################

def get_best_synset_pair(word_1, word_2):
    """ 
    Choose the pair with highest path similarity among all pairs. 
    Mimics pattern-seeking behavior of humans.
    """
    max_sim = -1.0
    synsets_1 = wn.synsets(word_1)
    synsets_2 = wn.synsets(word_2)
    if len(synsets_1) == 0 or len(synsets_2) == 0:
        return None, None
    else:
        max_sim = -1.0
        best_pair = None, None
        for synset_1 in synsets_1:
            for synset_2 in synsets_2:
                sim = wn.path_similarity(synset_1, synset_2)
                if sim is not None and sim > max_sim:
                    if sim > max_sim:
                        max_sim = sim
                        best_pair = synset_1, synset_2
        return best_pair

def length_dist(synset_1, synset_2):
    """
    Return a measure of the length of the shortest path in the semantic
    ontology (Wordnet in our case as well as the paper's) between two
    synsets.
    """
    l_dist = sys.maxsize
    if synset_1 is None or synset_2 is None:
        return 0.0
    if synset_1 == synset_2:
        # if synset_1 and synset_2 are the same synset return 0
        l_dist = 0.0
    else:
        wset_1 = set([str(x.name()) for x in synset_1.lemmas()])
        wset_2 = set([str(x.name()) for x in synset_2.lemmas()])
        if len(wset_1.intersection(wset_2)) > 0:
            # if synset_1 != synset_2 but there is word overlap, return 1.0
            l_dist = 1.0
        else:
            # just compute the shortest path between the two
            l_dist = synset_1.shortest_path_distance(synset_2)
            if l_dist is None:
                l_dist = 0.0
    # normalize path length to the range [0,1]
    return math.exp(-ALPHA * l_dist) 

def hierarchy_dist(synset_1, synset_2):
    """
    Return a measure of depth in the ontology to model the fact that 
    nodes closer to the root are broader and have less semantic similarity
    than nodes further away from the root.
    """
    h_dist = sys.maxsize
    if synset_1 is None or synset_2 is None: 
        return h_dist
    if synset_1 == synset_2:
        # return the depth of one of synset_1 or synset_2
        h_dist = max([x[1] for x in synset_1.hypernym_distances()])
    else:
        # find the max depth of least common subsumer
        hypernyms_1 = {x[0]:x[1] for x in synset_1.hypernym_distances()}
        hypernyms_2 = {x[0]:x[1] for x in synset_2.hypernym_distances()}
        lcs_candidates = set(hypernyms_1.keys()).intersection(
            set(hypernyms_2.keys()))
        if len(lcs_candidates) > 0:
            lcs_dists = []
            for lcs_candidate in lcs_candidates:
                lcs_d1 = 0
                if lcs_candidate in hypernyms_1:
                    lcs_d1 = hypernyms_1[lcs_candidate]
                lcs_d2 = 0
                if lcs_candidate in hypernyms_2:
                    lcs_d2 = hypernyms_2[lcs_candidate]
                lcs_dists.append(max([lcs_d1, lcs_d2]))
            h_dist = max(lcs_dists)
        else:
            h_dist = 0
    return ((math.exp(BETA * h_dist) - math.exp(-BETA * h_dist)) / 
        (math.exp(BETA * h_dist) + math.exp(-BETA * h_dist)))
    
def word_similarity(word_1, word_2):
    synset_pair = get_best_synset_pair(word_1, word_2)
    return (length_dist(synset_pair[0], synset_pair[1]) * 
        hierarchy_dist(synset_pair[0], synset_pair[1]))

In [452]:
######################### sentence similarity ##########################

def most_similar_word(word, word_set):
    """
    Find the word in the joint word set that is most similar to the word
    passed in. We use the algorithm above to compute word similarity between
    the word and each word in the joint word set, and return the most similar
    word and the actual similarity value.
    """
    max_sim = -1.0
    sim_word = ""
    for ref_word in word_set:
        sim = word_similarity(word, ref_word)
        if sim > max_sim:
            max_sim = sim
            sim_word = ref_word
        return sim_word, max_sim
    
def info_content(lookup_word):
    """
    Uses the Brown corpus available in NLTK to calculate a Laplace
    smoothed frequency distribution of words, then uses this information
    to compute the information content of the lookup_word.
    """
    global N
    if N == 0:
        # poor man's lazy evaluation
        for sent in brown.sents():
            for word in sent:
                word = word.lower()
                if word not in brown_freqs:
                    brown_freqs[word] = 0
                brown_freqs[word] = brown_freqs[word] + 1
                N = N + 1
    lookup_word = lookup_word.lower()
    n = 0 if lookup_word not in brown_freqs else brown_freqs[lookup_word]
    return 1.0 - (math.log(n + 1) / math.log(N + 1))
    
def semantic_vector(words, joint_words, info_content_norm):
    """
    Computes the semantic vector of a sentence. The sentence is passed in as
    a collection of words. The size of the semantic vector is the same as the
    size of the joint word set. The elements are 1 if a word in the sentence
    already exists in the joint word set, or the similarity of the word to the
    most similar word in the joint word set if it doesn't. Both values are 
    further normalized by the word's (and similar word's) information content
    if info_content_norm is True.
    """
    sent_set = set(words)
    semvec = np.zeros(len(joint_words))
    i = 0
    for joint_word in joint_words:
        if joint_word in sent_set:
            # if word in union exists in the sentence, s(i) = 1 (unnormalized)
            semvec[i] = 1.0
            if info_content_norm:
                semvec[i] = semvec[i] * math.pow(info_content(joint_word), 2)
        else:
            # find the most similar word in the joint set and set the sim value
            sim_word, max_sim = most_similar_word(joint_word, sent_set)
            semvec[i] = PHI if max_sim > PHI else 0.0
            if info_content_norm:
                semvec[i] = semvec[i] * info_content(joint_word) * info_content(sim_word)
        i = i + 1
    return semvec                
            
def semantic_similarity(sentence_1, sentence_2, info_content_norm):
    """
    Computes the semantic similarity between two sentences as the cosine
    similarity between the semantic vectors computed for each sentence.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = set(words_1).union(set(words_2))
    vec_1 = semantic_vector(words_1, joint_words, info_content_norm)
    vec_2 = semantic_vector(words_2, joint_words, info_content_norm)
    return np.dot(vec_1, vec_2.T) / (np.linalg.norm(vec_1) * np.linalg.norm(vec_2))

In [416]:
######################### word order similarity ##########################

def word_order_vector(words, joint_words, windex):
    """
    Computes the word order vector for a sentence. The sentence is passed
    in as a collection of words. The size of the word order vector is the
    same as the size of the joint word set. The elements of the word order
    vector are the position mapping (from the windex dictionary) of the 
    word in the joint set if the word exists in the sentence. If the word
    does not exist in the sentence, then the value of the element is the 
    position of the most similar word in the sentence as long as the similarity
    is above the threshold ETA.
    """
    wovec = np.zeros(len(joint_words))
    i = 0
    wordset = set(words)
    for joint_word in joint_words:
        if joint_word in wordset:
            # word in joint_words found in sentence, just populate the index
            wovec[i] = windex[joint_word]
        else:
            # word not in joint_words, find most similar word and populate
            # word_vector with the thresholded similarity
            sim_word, max_sim = most_similar_word(joint_word, wordset)
            if max_sim > ETA:
                wovec[i] = windex[sim_word]
            else:
                wovec[i] = 0
        i = i + 1
    return wovec

def word_order_similarity(sentence_1, sentence_2):
    """
    Computes the word-order similarity between two sentences as the normalized
    difference of word order between the two sentences.
    """
    words_1 = nltk.word_tokenize(sentence_1)
    words_2 = nltk.word_tokenize(sentence_2)
    joint_words = list(set(words_1).union(set(words_2)))
    windex = {x[1]: x[0] for x in enumerate(joint_words)}
    r1 = word_order_vector(words_1, joint_words, windex)
    r2 = word_order_vector(words_2, joint_words, windex)
    return 1.0 - (np.linalg.norm(r1 - r2) / np.linalg.norm(r1 + r2))

In [417]:
######################### overall similarity ##########################

def similarity(sentence_1, sentence_2, info_content_norm):
    """
    Calculate the semantic similarity between two sentences. The last 
    parameter is True or False depending on whether information content
    normalization is desired or not.
    """
    return DELTA * semantic_similarity(sentence_1, sentence_2, info_content_norm) + \
        (1.0 - DELTA) * word_order_similarity(sentence_1, sentence_2)

In [453]:
######################### main / test ##########################

# the results of the algorithm are largely dependent on the results of 
# the word similarities, so we should test this first...
# word_pairs = [
#   ["asylum", "fruit", 0.21],
#   ["autograph", "shore", 0.29],
#   ["autograph", "signature", 0.55],
#   ["automobile", "car", 0.64],
#   ["bird", "woodland", 0.33],
#   ["boy", "rooster", 0.53],
#   ["boy", "lad", 0.66],
#   ["boy", "sage", 0.51],
#   ["cemetery", "graveyard", 0.73],
#   ["coast", "forest", 0.36],
#   ["coast", "shore", 0.76],
#   ["cock", "rooster", 1.00],
#   ["cord", "smile", 0.33],
#   ["cord", "string", 0.68],
#   ["cushion", "pillow", 0.66],
#   ["forest", "graveyard", 0.55],
#   ["forest", "woodland", 0.70],
#   ["furnace", "stove", 0.72],
#   ["glass", "tumbler", 0.65],
#   ["grin", "smile", 0.49],
#   ["gem", "jewel", 0.83],
#   ["hill", "woodland", 0.59],
#   ["hill", "mound", 0.74],
#   ["implement", "tool", 0.75],
#   ["journey", "voyage", 0.52],
#   ["magician", "oracle", 0.44],
#   ["magician", "wizard", 0.65],
#   ["midday", "noon", 1.0],
#   ["oracle", "sage", 0.43],
#   ["serf", "slave", 0.39]
# ]
# for word_pair in word_pairs:
#     print("%s\t%s\t%.2f\t%.2f" % (word_pair[0], word_pair[1], word_pair[2],
#                                   word_similarity(word_pair[0], word_pair[1])))





sentence_pairs = [
    ["I like that bachelor.", "I like that unmarried man.", 0.561],
    ["John is very nice.", "Is John very nice?", 0.977],
    ["Red alcoholic drink.", "A bottle of wine.", 0.585],
    ["Red alcoholic drink.", "Fresh orange juice.", 0.611],
    ["Red alcoholic drink.", "An English dictionary.", 0.0],
    ["Red alcoholic drink.", "Fresh apple juice.", 0.420],
    ["A glass of cider.", "A full cup of apple juice.", 0.678],
    ["It is a dog.", "That must be your dog.", 0.739],
    ["It is a dog.", "It is a log.", 0.623],
    ["It is a dog.", "It is a pig.", 0.790],
    ["Dogs are animals.", "They are common pets.", 0.738],
    ["Canis familiaris are animals.", "Dogs are common pets.", 0.362],
    ["I have a pen.", "Where do you live?", 0.0],
    ["I have a pen.", "Where is ink?", 0.129],
    ["I have a hammer.", "Take some nails.", 0.508],
    ["I have a hammer.", "Take some apples.", 0.121]
]
for sent_pair in sentence_pairs:
    print("%s\t%s\t%.3f\t%.3f\t%.3f" % (sent_pair[0], sent_pair[1], sent_pair[2], 
        similarity(sent_pair[0], sent_pair[1], False),
        similarity(sent_pair[0], sent_pair[1], True)))

I like that bachelor.	I like that unmarried man.	0.561	0.788	0.319
John is very nice.	Is John very nice?	0.977	0.615	0.903
Red alcoholic drink.	A bottle of wine.	0.585	0.256	0.069
Red alcoholic drink.	Fresh orange juice.	0.611	0.268	0.058
Red alcoholic drink.	An English dictionary.	0.000	0.284	0.074
Red alcoholic drink.	Fresh apple juice.	0.420	0.252	0.042
A glass of cider.	A full cup of apple juice.	0.678	0.626	0.334
It is a dog.	That must be your dog.	0.739	0.422	0.682
It is a dog.	It is a log.	0.623	0.808	0.257
It is a dog.	It is a pig.	0.790	0.784	0.188
Dogs are animals.	They are common pets.	0.738	0.420	0.078
Canis familiaris are animals.	Dogs are common pets.	0.362	0.394	0.070
I have a pen.	Where do you live?	0.000	0.101	0.120
I have a pen.	Where is ink?	0.129	0.100	0.046
I have a hammer.	Take some nails.	0.508	0.256	0.070
I have a hammer.	Take some apples.	0.121	0.293	0.138


In [125]:

def clean(path, text0, stopwords):
    ##returns a list of lists, with words in it.
    with open(path+text0) as txt:
        data = txt.read()

        # data = re.sub('\\w+\\set\\sal', '<UNK>', data)
        data = re.sub('\\w+\\sand\\s+\\w+\s\\(\\d{4}\\)', '<UNK>', data)
        data = re.sub('[A-Z][a-z]*\s+\\(\\d{4}\\)', '<UNK>', data)
        data = re.sub('\\([^)]*\\)', '<UNK>', data)
        data = re.sub('[A-Z][a-z]*\\set\\sal.', '<UNK>', data)
        data = re.sub('\s+[.]', '.', data)
        data = re.sub('\s+[,]', ',', data)
        data = data.replace(u'\xa0', u' ')
        data = data.split('\n')
        data = [n.strip().split('.') for n in data]
        data = [[z.strip().split(' ') for z in n] for n in data]
        data = [[z for z in n if len(z) > 5] for n in data if len(n) > 0]
        data = [n for n in data if len(n) > 0]
        data = [[n for sublist in l for n in sublist] for l in data]
        data = [[n.lower() for n in l if not re.search('\\d', n)] for l in data]
        data = [[n for n in l if re.search('[a-zA-Z]', n)] for l in data] ####TODO check
        data = [[n for n in l if n not in stopwords] for l in data]
        data = [[n for n in l if '<unk>' not in n] for l in data]
        data = [[n for n in l if len(n)>1] for l in data]
        data = [[n.replace(',','') for n in l] for l in data]
        data = [[n.replace("'s",'') for n in l] for l in data]
        # data = [[n.split('-') for n in l] for l in data]
        # data = [[n for n in l if not re.search('\\', n)] for l in data]
        txt.close()
    return data

papers = [n for n in os.listdir(os.path.join(root, 'papers')) if n.endswith('txt')]
test = random.choice(papers)

In [126]:
os.path.join(os.path.join(root, 'papers'),test)

'C:\\Users\\ifue3702\\Documents\\GitHub\\Litho\\papers\\10.1016-j.carres.2004.12.026.txt'

In [128]:
texto = open(os.path.join(os.path.join(root, 'papers'),test))
texto.read()
#     data = txt.read()
#     print(data)
texto.close()